# Learning Style Classification V3 (CORRECTED PIPELINE)

## 🎯 Objective
This notebook implements a scientifically rigorous pipeline to fix the **Methodological Flaws** identified in previous versions (Data Leakage via Global Imputation & Global Oversampling).

## 🛡️ Corrected Workflow "The Gold Standard"
We strictly adhere to the rule: **"The Test Set must remain unseen during all preprocessing steps."**

1.  **Split**: 10-Fold Stratified Nested Cross-Validation.
2.  **Inside Each Fold (Training Data Only)**:
    -   **Imputation**: Fit on Train, Transform on Train.
    -   **Oversampling**: Creating synthetic samples based ONLY on Train.
    -   **Model Training**: Train on the clean, oversampled Train data.
3.  **Evaluation (Test Data Only)**:
    -   **Imputation**: Transform Test using statistics from Train.
    -   **Prediction**: Predict on the **original, unmodified** Test data.
    -   **Score**: Calculate metrics.

## 🔬 Experimental Design (Grid Search)
We let the data decide the absolute best combination of preprocessing and modeling.
- **Imputers**: Zero, Mean, Median, MICE.
- **Samplers**: RandomOverSampler, SMOTE, ADASYN, None.
- **Models**: Random Forest (Baseline), XGBoost, SVM.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from copy import deepcopy

# Sklearn Core
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.metrics import f1_score, make_scorer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator, TransformerMixin

# Imbalanced-Learn (The Hero of this Notebook)
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN

# Models
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.svm import SVC

# Utils
from output_paths import get_data_output_path, get_model_output_path
import joblib

warnings.filterwarnings('ignore')
np.random.seed(42)
print("✅ Libraries Loaded Successfully")

## 1. Data Loading (RAW DATA ONLY)
> **CRITICAL**: We load the raw CSV files (`dfjadi-simplified.csv` & `mhs_grouping_by_material_type.csv`). We do **NOT** load any pre-processed or oversampled files from previous notebooks to ensure purity.

In [ ]:
# Load Raw Data
df_styles = pd.read_csv('dataset/dfjadi-simplified - dfjadi-simplified.csv')
df_time = pd.read_csv('dataset/mhs_grouping_by_material_type.csv')

print(f"Raw Learning Styles Shape: {df_styles.shape}")
print(f"Raw Time Logs Shape: {df_time.shape}")

# Merge Logic (Re-implementing basic cleaning from EDA)
# 1. Standardize IDs
df_styles['NIM'] = df_styles['NIM'].astype(str).str.upper().str.strip()
df_time['NPM'] = df_time['NPM'].astype(str).str.upper().str.strip()

# 2. Merge
df_merged = pd.merge(df_styles, df_time, left_on='NIM', right_on='NPM', how='inner')

# 3. Features & Targets
# We use the 3 best time features identified previously
TIME_FEATURES = ['time_materials_video', 'time_materials_document', 'time_materials_article']
# Target Preprocessing
def parse_labels(row):
    labels = []
    # Map raw values (e.g. "Reflektif Rendah") to simplified labels ("Reflektif")
    # Mapping based on "Pemrosesan" and "Input" columns in raw data
    
    pemrosesan = str(row['Pemrosesan'])
    if 'Aktif' in pemrosesan:
        labels.append('Aktif')
    elif 'Reflektif' in pemrosesan:
        labels.append('Reflektif')
        
    input_style = str(row['Input'])
    if 'Visual' in input_style:
        labels.append('Visual')
    elif 'Verbal' in input_style:
        labels.append('Verbal')
        
    return labels

df_merged['labels'] = df_merged.apply(parse_labels, axis=1)

# MultiLabel Encoding
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df_merged['labels'])
X = df_merged[TIME_FEATURES]

print(f"\nFinal Dataset for Modeling: {X.shape}")
print(f"Features: {X.columns.tolist()}")
print(f"Classes: {mlb.classes_}")

# Check for NaN in X (This is what we will impute dynamically)
print("\nMissing Values in X:")
print(X.isnull().sum())

## 2. Pipeline Components
We define custom wrappers to make `IterativeImputer` (MICE) and Multi-Label classification compatible with `GridSearchCV`.

In [ ]:
# Custom Imputer Wrapper to handle 'constant' strategy requiring fill_value
class SmartImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='median', fill_value=0):
        self.strategy = strategy
        self.fill_value = fill_value
        self.imputer = None

    def fit(self, X, y=None):
        if self.strategy == 'mice':
            self.imputer = IterativeImputer(max_iter=10, random_state=42)
        elif self.strategy == 'constant':
            self.imputer = SimpleImputer(strategy='constant', fill_value=self.fill_value)
        else:
            self.imputer = SimpleImputer(strategy=self.strategy)
        self.imputer.fit(X, y)
        return self

    def transform(self, X):
        return self.imputer.transform(X)

# Custom MultiLabel Oversampler Wrapper
# Since most samplers don't support multi-label targets natively (y is 2D array),
# We usually use a specialized multi-label sampler like MLSMOTE, OR 
# for simplicity in this robust pipeline, we can transform y to 'powerset' classes for sampling, then back.
# This prevents the 'ValueError: y should be a 1d array' for standard SMOTE.

from sklearn.preprocessing import LabelEncoder

class MultiLabelSamplerWrapper(BaseEstimator):
    def __init__(self, sampler=None):
        self.sampler = sampler

    def fit_resample(self, X, y):
        if self.sampler is None:
            return X, y
        
        # Convert Multi-Label (Binary Indicator) to Multi-Class (Powerset) for Sampling
        # e.g., [1, 0, 1] -> "Class_A_C"
        y_str = [''.join(map(str, row)) for row in y]
        
        # Sample using the powerset classes
        X_res, y_str_res = self.sampler.fit_resample(X, y_str)
        
        # Convert back to Multi-Label
        # We need a robust way to map back.
        # Since we just need to confirm pipeline mechanics, let's use a simpler approach for now:
        # We will use the 'LP' (Label Powerset) transformation implicitly by mapping unique string patterns back to arrays.
        
        # Create a mapping from str back to array based on original data
        unique_patterns = np.unique(y_str)
        mapping = {}
        # Find one example for each pattern in original y to create map
        for i, pat in enumerate(y_str):
            if pat not in mapping:
                mapping[pat] = y[i]
        
        # Reconstruct y_res
        y_res = np.array([mapping[pat] for pat in y_str_res])
        
        return X_res, y_res

# However, creating a fully custom sampler compatible with ImbPipeline is complex.
# A cleaner approach for this notebook is to rely on 'MultiOutputClassifier' handling the classification,
# but 'ImbPipeline' needs y to be resamplable.
# Standard SMOTE does not support multi-label.
# We will use the 'Label Powerset' approach for SMOTE within the pipeline manually if needed, 
# or use a simplified approach: Independent sampling (rarely optimal) vs LP sampling.

# Let's define the Grid Search Strategy properly.


## 3. The Grand Experiment (Nested CV Grid Search)
We define the **Search Space** for the pipeline.
- `imputer__strategy`: ['mean', 'median', 'constant', 'mice']
- `sampler__sampler`: [RandomOverSampler, SMOTE, ADASYN, None]


In [ ]:
# Define the Base Pipeline Structure
# Note: We wrap the Classifier in MultiOutputClassifier for multi-label support AFTER sampling.
# But Sampler needs to handle multi-label y. We use the wrapper defined above.

def get_pipeline(classifier):
    return ImbPipeline([
        ('imputer', SmartImputer()),
        ('scaler', StandardScaler()),
        # We use our custom wrapper to enable SMOTE on Multi-Label data via Label Powerset method
        ('sampler', MultiLabelSamplerWrapper()), 
        ('classifier', MultiOutputClassifier(classifier))
    ])

# Custom Safe Wrappers for SMOTE/ADASYN to handle n_samples < 2
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
class SafeSMOTE(SMOTE):
    def fit_resample(self, X, y):
        # check min class count
        from collections import Counter
        counts = Counter(y)
        min_samples = min(counts.values())
        if min_samples < 2:
            return RandomOverSampler(random_state=42).fit_resample(X, y)
        return super().fit_resample(X, y)

class SafeADASYN(ADASYN):
    def fit_resample(self, X, y):
         # check min class count
        from collections import Counter
        counts = Counter(y)
        min_samples = min(counts.values())
        if min_samples < 2:
            return RandomOverSampler(random_state=42).fit_resample(X, y)
        return super().fit_resample(X, y)

# Define the GRID
param_grid_template = {
    'imputer__strategy': ['mean', 'median', 'constant', 'mice'],
    'sampler__sampler': [
        None,
        RandomOverSampler(random_state=42),
        SafeSMOTE(random_state=42, k_neighbors=1), # Safe wrapper with fallback
        SafeADASYN(random_state=42, n_neighbors=1) # Safe wrapper with fallback
    ]
}

# We will run this for Random Forest first as it's our best candidate
rf_params = param_grid_template.copy()
rf_params.update({
    'classifier__estimator__n_estimators': [100, 200],
    'classifier__estimator__max_depth': [10, None]
})

print("Grid Search Space Defined.")

## 4. Execution: Running the Nested CV
This process will take some time as it trains hundreds of models.
- **Outer CV**: 10 Folds (To estimate realistic performance).
- **Inner CV**: 3 Folds (To select best hyperparameters/strategies).


In [ ]:
# Metrics
scoring = 'f1_macro'

# 1. Random Forest Experiment
print("🚀 Starting Experiment 1: Random Forest + Strategy Search...")

rf_pipeline = get_pipeline(RandomForestClassifier(random_state=42))

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
# Inner CV: Use KFold instead of StratifiedKFold to avoid 'multilabel-indicator' error in GridSearchCV
# Since standard StratifiedKFold doesn't support multi-label targets directly.
from sklearn.model_selection import KFold
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Since StratifiedKFold expects 1D y array for splitting the OUTER loop, we use Label Powerset transformation
y_str_strat = [''.join(map(str, row)) for row in y]

grid_search = GridSearchCV(
    estimator=rf_pipeline,
    param_grid=rf_params,
    cv=inner_cv,
    scoring=scoring,
    n_jobs=-1,
    verbose=1,
    error_score='raise' # Debugging
)

# Run Nested CV
# We manually loop outer folds to easily track best params per fold
outer_results = []
best_configs = []

for fold, (train_idx, test_idx) in enumerate(outer_cv.split(X, y_str_strat)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    # Fit Grid Search (Inner CV)
    grid_search.fit(X_train, y_train)
    
    # Best Model from Inner CV
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    best_score_inner = grid_search.best_score_
    
    # Evaluate on Outer Test Set (The Moment of Truth)
    y_pred = best_model.predict(X_test)
    test_score = f1_score(y_test, y_pred, average='macro')
    
    print(f"Fold {fold+1}/10 | Test F1: {test_score:.4f} | Best Imputer: {best_params['imputer__strategy']} | Best Sampler: {best_params['sampler__sampler']}")
    
    outer_results.append(test_score)
    best_configs.append(best_params)

print(f"\n✅ Final Corrected F1-Macro (Avg): {np.mean(outer_results):.4f} (+/- {np.std(outer_results):.4f})")


## 5. Analysis & Choice of Best Strategy
We analyze the `best_configs` list to see which preprocessing strategy won the most often across the 10 folds. This gives us the empirically "Best" strategy.


In [ ]:
# Analyze winning strategies
from collections import Counter

imputer_wins = Counter([cfg['imputer__strategy'] for cfg in best_configs])
sampler_wins = Counter([str(cfg['sampler__sampler']) for cfg in best_configs])

print("🏆 Winning Imputation Strategies:")
for Strat, Count in imputer_wins.most_common():
    print(f"  - {Strat}: {Count} folds")

print("\n🏆 Winning Oversampling Techniques:")
for Strat, Count in sampler_wins.most_common():
    print(f"  - {Strat}: {Count} folds")
